In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DRC_table = pd.read_excel('DRCtableAUC_merge_DepmapID.xlsx')
DRC_table.head()

,Cell name,Depmap ID,"AUC, DEG-35","AUC, DEG-35 sigfig 4"
0,143B,ACH-001001,0.738025,0.7380
1,22RV1,ACH-000956,0.543824,0.5438
2,2313287,ACH-000948,0.362258,0.3623
3,253J,ACH-000011,0.811752,0.8118
4,253JBV,ACH-000026,0.713113,0.7131


In [3]:
df_SomaticMT = pd.read_csv('OmicsSomaticMutations.csv', low_memory=False)

In [4]:
df_SomaticMT = df_SomaticMT.rename(columns={'ModelID': 'Depmap ID'})
df_SomaticMT = df_SomaticMT.merge(DRC_table, how='inner')
df_SomaticMT.head()

,Chrom,Pos,Ref,Alt,AF,DP,RefCount,AltCount,GT,PS,...,GwasDisease,GwasPmID,GtexGene,ProveanPrediction,Rescue,Depmap ID,EntrezGeneID,Cell name,"AUC, DEG-35","AUC, DEG-35 sigfig 4"
0,chr1,818203,G,A,0.240,27,21,6,0/1,NaN,...,NaN,NaN,NaN,NaN,False,ACH-000062,400728.0,RERFLCMS,0.834513,0.8345
1,chr1,24469099,A,G,0.978,44,0,44,1|1,NaN,...,NaN,NaN,NaN,Neutral,False,ACH-000062,57185.0,RERFLCMS,0.834513,0.8345
2,chr1,34899671,G,C,0.155,30,26,4,0/1,NaN,...,NaN,NaN,NaN,Neutral,False,ACH-000062,58512.0,RERFLCMS,0.834513,0.8345
3,chr1,35390086,C,T,0.150,41,35,6,0/1,NaN,...,NaN,NaN,NaN,NaN,False,ACH-000062,9202.0,RERFLCMS,0.834513,0.8345
4,chr1,48776614,G,T,0.799,43,8,35,0/1,NaN,...,NaN,NaN,NaN,Neutral,False,ACH-000062,79656.0,RERFLCMS,0.834513,0.8345


In [6]:
df_SomaticMT.to_csv('Results/SomaticMT/Take2/df_SomaticMT.csv')

In [7]:
MT_gene_avg_auc = df_SomaticMT.groupby('HugoSymbol').agg({
    'AUC, DEG-35 sigfig 4': 'mean',
    'Cell name': lambda x: ', '.join(map(str, x)),
    'HgncName': 'first'
}).reset_index()

MT_gene_avg_auc.head()

,HugoSymbol,"AUC, DEG-35 sigfig 4",Cell name,HgncName
0,A1BG,0.726000,"NCIH1435, MFE319, WM2664, HT115, KYSE450, HGC2...",alpha-1-B glycoprotein
1,A1CF,0.698164,"22RV1, AN3CA, HT115, NCIH650, NCIH1092, KATOII...",APOBEC1 complementation factor
2,A2M,0.671580,"LS180, NCIH1435, MFE319, AN3CA, RKO, GP2D, GP2...",alpha-2-macroglobulin
3,A2ML1,0.715028,"22RV1, HEC1A, NCIH1435, WM2664, WM2664, VMCUB1...",alpha-2-macroglobulin like 1
4,A3GALT2,0.772728,"SNU407, LS180, MFE296, MFE319, SNUC2A, KYSE450...",alpha 1%3B3-galactosyltransferase 2


In [8]:
MT_gene_avg_auc.to_csv('Results/SomaticMT//Take2/MT-Gene_AvgAUC.csv')

In [9]:
# Step 1: Create a new DataFrame to store the results
WT_gene_avg_auc = pd.DataFrame(columns=['HugoSymbol', 'Mean AUC'])

# Step 2: Iterate over each row in MT_gene_avg_auc
for index, row in MT_gene_avg_auc.iterrows():
    # Split the cell names and convert to a set
    cell_names = set(row['Cell name'].split(', '))
    
    # Subtract the cell names from the total list of cell names
    new_cell_names = set(DRC_table['Cell name']).difference(cell_names)
    
    # Filter DRC to include only the rows with cell names from the new list of cell names
    new_DRC = DRC_table[DRC_table['Cell name'].isin(new_cell_names)]
    
    # Calculate the mean AUC for the remaining cell names
    mean_auc = new_DRC['AUC, DEG-35 sigfig 4'].mean()
    
    # Convert all elements in new_cell_names to strings before joining
    new_cell_names = [str(cell) for cell in new_cell_names]
    
    # Append the results to new_gene_avg_auc
    #WT_gene_avg_auc = WT_gene_avg_auc.append({'HugoSymbol': row['HugoSymbol'], 'Mean AUC': mean_auc, 'WT Cell names': ', '.join(new_cell_names)}, ignore_index=True)
    new_row = {'HugoSymbol': row['HugoSymbol'], 'Mean AUC': mean_auc, 'WT Cell names': ', '.join(new_cell_names)}

    # Convert the new row into a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with WT_gene_avg_auc
    WT_gene_avg_auc = pd.concat([WT_gene_avg_auc, new_row_df], ignore_index=True)


In [10]:
WT_gene_avg_auc.head()

,HugoSymbol,Mean AUC,WT Cell names
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,..."
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,..."
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,..."
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,..."
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,..."


In [11]:
WT_gene_avg_auc.to_excel('Results/SomaticMT/Take2/WT_gene_avg_AUC_final.xlsx')

In [12]:
# Merge the two DataFrames on 'HugoSymbol'
merged_df = pd.merge(WT_gene_avg_auc, MT_gene_avg_auc, on='HugoSymbol', suffixes=('_WT', '_MT'))

# Calculate the difference of the mean AUCs
merged_df['AUC_difference_MT-WT'] = merged_df['AUC, DEG-35 sigfig 4'] - merged_df['Mean AUC']

difference_df = merged_df[['HugoSymbol', 'AUC_difference_MT-WT']]

In [13]:
difference_df.head()

,HugoSymbol,AUC_difference_MT-WT
0,A1BG,-0.021706
1,A1CF,-0.051185
2,A2M,-0.078305
3,A2ML1,-0.034266
4,A3GALT2,0.026215


In [14]:
difference_df.to_excel('Results/SomaticMT/Take2/AUC_difference_Mt-Wt.xlsx')

In [15]:
def get_auc(cell_names):
    return DRC_table[DRC_table['Cell name'].isin(cell_names)]['AUC, DEG-35 sigfig 4'].tolist()

WT_gene_avg_auc2 = WT_gene_avg_auc
WT_gene_avg_auc2['AUC values'] = WT_gene_avg_auc2['WT Cell names'].apply(lambda x: get_auc(x.split(', ')))

MT_gene_avg_auc2 = MT_gene_avg_auc
MT_gene_avg_auc['AUC values'] = MT_gene_avg_auc['Cell name'].apply(lambda x: get_auc(x.split(', ')))

In [16]:
WT_gene_avg_auc2.head()

,HugoSymbol,Mean AUC,WT Cell names,AUC values
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712..."
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7712, 0.7718..."
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712..."
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7718, 0.5395..."
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712..."


In [17]:
All_cell_MTvsWT = WT_gene_avg_auc2.merge(MT_gene_avg_auc2, how = 'inner', on = 'HugoSymbol', suffixes=('_WT','_MT'))

In [18]:
All_cell_MTvsWT.head()

,HugoSymbol,Mean AUC,WT Cell names,AUC values_WT,"AUC, DEG-35 sigfig 4",Cell name,HgncName,AUC values_MT
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.726000,"NCIH1435, MFE319, WM2664, HT115, KYSE450, HGC2...",alpha-1-B glycoprotein,"[0.6903, 0.9147, 0.6259, 0.7974, 0.7072, 0.522..."
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7712, 0.7718...",0.698164,"22RV1, AN3CA, HT115, NCIH650, NCIH1092, KATOII...",APOBEC1 complementation factor,"[0.5438, 0.7057, 0.6913, 0.8033, 0.6547, 0.574..."
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.671580,"LS180, NCIH1435, MFE319, AN3CA, RKO, GP2D, GP2...",alpha-2-macroglobulin,"[0.8033, 0.8347, 0.8042, 0.2974, 0.7529, 0.662..."
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7718, 0.5395...",0.715028,"22RV1, HEC1A, NCIH1435, WM2664, WM2664, VMCUB1...",alpha-2-macroglobulin like 1,"[0.5438, 0.7712, 0.8492, 0.8033, 0.8145, 0.804..."
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.772728,"SNU407, LS180, MFE296, MFE319, SNUC2A, KYSE450...",alpha 1%3B3-galactosyltransferase 2,"[0.8092, 0.7366, 0.8457, 0.4126, 0.7437, 0.503..."


In [19]:
All_cell_MTvsWT = All_cell_MTvsWT.rename(columns={
    'Mean AUC':'Mean AUC_WT', 
    'WT Cell names':'Cell names_WT',
    'AUC, DEG-35':'Mean AUC_MT',
    'Cell name':'Cell names_MT'})

In [20]:
All_cell_MTvsWT.head()

,HugoSymbol,Mean AUC_WT,Cell names_WT,AUC values_WT,"AUC, DEG-35 sigfig 4",Cell names_MT,HgncName,AUC values_MT
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.726000,"NCIH1435, MFE319, WM2664, HT115, KYSE450, HGC2...",alpha-1-B glycoprotein,"[0.6903, 0.9147, 0.6259, 0.7974, 0.7072, 0.522..."
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7712, 0.7718...",0.698164,"22RV1, AN3CA, HT115, NCIH650, NCIH1092, KATOII...",APOBEC1 complementation factor,"[0.5438, 0.7057, 0.6913, 0.8033, 0.6547, 0.574..."
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.671580,"LS180, NCIH1435, MFE319, AN3CA, RKO, GP2D, GP2...",alpha-2-macroglobulin,"[0.8033, 0.8347, 0.8042, 0.2974, 0.7529, 0.662..."
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7718, 0.5395...",0.715028,"22RV1, HEC1A, NCIH1435, WM2664, WM2664, VMCUB1...",alpha-2-macroglobulin like 1,"[0.5438, 0.7712, 0.8492, 0.8033, 0.8145, 0.804..."
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.772728,"SNU407, LS180, MFE296, MFE319, SNUC2A, KYSE450...",alpha 1%3B3-galactosyltransferase 2,"[0.8092, 0.7366, 0.8457, 0.4126, 0.7437, 0.503..."


In [21]:
All_cell_MTvsWT = All_cell_MTvsWT.merge(difference_df, how = 'left', on = 'HugoSymbol')

In [22]:
All_cell_MTvsWT.head()

,HugoSymbol,Mean AUC_WT,Cell names_WT,AUC values_WT,"AUC, DEG-35 sigfig 4",Cell names_MT,HgncName,AUC values_MT,AUC_difference_MT-WT
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.726000,"NCIH1435, MFE319, WM2664, HT115, KYSE450, HGC2...",alpha-1-B glycoprotein,"[0.6903, 0.9147, 0.6259, 0.7974, 0.7072, 0.522...",-0.021706
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7712, 0.7718...",0.698164,"22RV1, AN3CA, HT115, NCIH650, NCIH1092, KATOII...",APOBEC1 complementation factor,"[0.5438, 0.7057, 0.6913, 0.8033, 0.6547, 0.574...",-0.051185
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.671580,"LS180, NCIH1435, MFE319, AN3CA, RKO, GP2D, GP2...",alpha-2-macroglobulin,"[0.8033, 0.8347, 0.8042, 0.2974, 0.7529, 0.662...",-0.078305
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7718, 0.5395...",0.715028,"22RV1, HEC1A, NCIH1435, WM2664, WM2664, VMCUB1...",alpha-2-macroglobulin like 1,"[0.5438, 0.7712, 0.8492, 0.8033, 0.8145, 0.804...",-0.034266
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.772728,"SNU407, LS180, MFE296, MFE319, SNUC2A, KYSE450...",alpha 1%3B3-galactosyltransferase 2,"[0.8092, 0.7366, 0.8457, 0.4126, 0.7437, 0.503...",0.026215


In [23]:
All_cell_MTvsWT.to_excel('Results/SomaticMT/Take2/All_cell_MTvsWT.xlsx')

In [24]:
from scipy.stats import levene

In [27]:
# Step 1: Iterate over each row in MT_gene_avg_auc
F_test = pd.DataFrame(columns=['HugoSymbol', 'stat', 'p_value'])

for index, row in All_cell_MTvsWT.iterrows():
    
    # Extract AUC values from the row
    auc1 = row['AUC values_WT']
    auc2 = row['AUC values_MT']
    gene = row['HugoSymbol']
    
    # Convert the lists of arrays into arrays of floating-point numbers
    auc_array1 = np.array(auc1, dtype=float)
    auc_array2 = np.array(auc2, dtype=float)
    
    # Filter out NaN values
    auc_array1 = auc_array1[~np.isnan(auc_array1)]
    auc_array2 = auc_array2[~np.isnan(auc_array2)]
    
    if len(auc_array1) == 0 or len(auc_array2) == 0:
        # If one of the arrays is empty after filtering NaN values, skip this row
        continue
    
    # Perform Levene's test for equal variances
    stat, p_value = levene(auc_array1, auc_array2)
    
    # F-test results
    new_row = {'HugoSymbol': gene, 'stat': stat, 'p_value': p_value}

    # Convert the new row into a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with F_test
    F_test = pd.concat([F_test, new_row_df], ignore_index=True)


In [28]:
F_test.head()

,HugoSymbol,stat,p_value
0,A1BG,0.600098,0.438866
1,A1CF,2.056289,0.152135
2,A2M,0.007698,0.930115
3,A2ML1,0.008504,0.926557
4,A3GALT2,0.029614,0.863430


In [29]:
All_cell_MTvsWT = All_cell_MTvsWT.merge(F_test, how='inner', on = 'HugoSymbol')
All_cell_MTvsWT = All_cell_MTvsWT.rename(columns={
    'stat':'F-Test_stat', 
    'p_value':'F-Test_p-value',
    })

In [30]:
def Ftest(i):
    if i > 0.05:
        return True
    return False

# Apply the function to create a new column
All_cell_MTvsWT['F-test'] = All_cell_MTvsWT['F-Test_p-value'].apply(Ftest)

In [31]:
All_cell_MTvsWT.to_excel('Results/SomaticMT/Take2/All_cell_MTvsWT_FTEST.xlsx')

In [32]:
from scipy.stats import ttest_ind
from scipy import stats

In [33]:
df_cleaned = All_cell_MTvsWT

In [36]:
# Initialize an empty list to store the counts
counts = []

# Iterate over the rows of the DataFrame
for index, row in df_cleaned.iterrows():
    # Count the number of elements in the list
    num_elements = len(row['AUC values_WT'])
    # Append the count to the list
    counts.append(num_elements)

# Add the counts as a new column to the DataFrame
df_cleaned['Num_of_WT'] = counts


In [37]:
# Initialize an empty list to store the counts
counts = []

# Iterate over the rows of the DataFrame
for index, row in df_cleaned.iterrows():
    # Count the number of elements in the list
    num_elements = len(row['AUC values_MT'])
    # Append the count to the list
    counts.append(num_elements)

# Add the counts as a new column to the DataFrame
df_cleaned['Num_of_MT'] = counts


In [38]:
# Filter rows with Num_Elements >= 3
df_cleaned_filtered = df_cleaned[df_cleaned['Num_of_WT'] > 3]
df_cleaned_filtered = df_cleaned[df_cleaned['Num_of_MT'] > 3]

In [39]:
df_cleaned_filtered.head()

,HugoSymbol,Mean AUC_WT,Cell names_WT,AUC values_WT,"AUC, DEG-35 sigfig 4",Cell names_MT,HgncName,AUC values_MT,AUC_difference_MT-WT,F-Test_stat,F-Test_p-value,F-test,Num_of_WT,Num_of_MT
0,A1BG,0.747706,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.726000,"NCIH1435, MFE319, WM2664, HT115, KYSE450, HGC2...",alpha-1-B glycoprotein,"[0.6903, 0.9147, 0.6259, 0.7974, 0.7072, 0.522...",-0.021706,0.600098,0.438866,True,550,14
1,A1CF,0.749349,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7712, 0.7718...",0.698164,"22RV1, AN3CA, HT115, NCIH650, NCIH1092, KATOII...",APOBEC1 complementation factor,"[0.5438, 0.7057, 0.6913, 0.8033, 0.6547, 0.574...",-0.051185,2.056289,0.152135,True,542,22
2,A2M,0.749885,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.671580,"LS180, NCIH1435, MFE319, AN3CA, RKO, GP2D, GP2...",alpha-2-macroglobulin,"[0.8033, 0.8347, 0.8042, 0.2974, 0.7529, 0.662...",-0.078305,0.007698,0.930115,True,537,27
3,A2ML1,0.749295,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.8118, 0.7131, 0.7764, 0.7718, 0.5395...",0.715028,"22RV1, HEC1A, NCIH1435, WM2664, WM2664, VMCUB1...",alpha-2-macroglobulin like 1,"[0.5438, 0.7712, 0.8492, 0.8033, 0.8145, 0.804...",-0.034266,0.008504,0.926557,True,529,35
4,A3GALT2,0.746513,"HS294T, NCIH446, PANC0327, HARA, HS683, ECC10,...","[0.738, 0.5438, 0.8118, 0.7131, 0.7764, 0.7712...",0.772728,"SNU407, LS180, MFE296, MFE319, SNUC2A, KYSE450...",alpha 1%3B3-galactosyltransferase 2,"[0.8092, 0.7366, 0.8457, 0.4126, 0.7437, 0.503...",0.026215,0.029614,0.863430,True,547,17


In [40]:
import pandas as pd
import numpy as np
from scipy import stats

# Step 1: Iterate over each row in MT_gene_avg_auc
T_test = pd.DataFrame(columns=['HugoSymbol','stat', 'p_value', 'DoF'])

# Iterate over rows of df_cleaned_filtered
for index, row in df_cleaned_filtered.iterrows():
    
    # Extract AUC values from the row and handle missing values
    auc1 = row['AUC values_WT']
    auc2 = row['AUC values_MT']
    gene = row['HugoSymbol']
    
    # Padding arrays to make them the same length
    max_length = max(len(auc1), len(auc2))
    auc1_padded = auc1 + [np.nan] * (max_length - len(auc1))
    auc2_padded = auc2 + [np.nan] * (max_length - len(auc2))
    
    data = {'WT': auc1_padded, 'MT': auc2_padded}
    df = pd.DataFrame(data)
    
    df['WT'] = pd.to_numeric(df['WT'], errors='coerce')
    df['MT'] = pd.to_numeric(df['MT'], errors='coerce')
    
    var = row['F-test']
    
    # Perform t-test
    if var:  # If F-test indicates equal variances
        stat, p_value = stats.ttest_ind(df['WT'], df['MT'], equal_var=True, nan_policy='omit')
    else:
        stat, p_value = stats.ttest_ind(df['WT'], df['MT'], equal_var=False, nan_policy='omit')
        
    # Calculate degrees of freedom
    DoF = max_length - 2
    
    # Create a new row with the t-test results
    new_row = {'HugoSymbol' : gene, 'stat': stat, 'p_value': p_value, 'DoF': DoF}
    
    # Convert the new row into a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new row DataFrame with T_test
    T_test = pd.concat([T_test, new_row_df], ignore_index=True)
    

In [41]:
df_cleaned_filtered = df_cleaned_filtered.merge(T_test, how='inner', on = 'HugoSymbol')
df_cleaned_filtered = df_cleaned_filtered.rename(columns={
    'stat':'T-Test_stat', 
    'p_value':'T-Test_p-value',
    })

In [42]:
df_cleaned_filtered.to_excel('Results/SomaticMT/Take2/All_cell_MTvsWT_TTEST.xlsx')